In [ ]:
import numpy as np
import matplotlib.pyplot as plt  
%matplotlib inline
# from IPython.display import display, HTML

from pykat import finesse
from pykat.commands import *
pykat.init_pykat_plotting(dpi=90)

from pprint import pprint as pprint

In [ ]:
basekat = finesse.kat()
basecode = """
# constants
const Pin 1M # laser power
const LX 4k # arm lengths
const LY 4k
const Lrecyc 1 # L = 1m, recycling cavities
# recycling cavities are overcoupled
const recycR1 0.8
const recycT1 0.2
const recycR2 0.9
const recycT2 0.1
# arm cavities are also?
const R1arm 0.9
const T1arm 0.1
const R2arm 1 # perfect mirror
const T2arm 0

# laser
l laser $Pin 0 nLaser
s slasertoPR 0 nLaser nPR0 # null length

# power recycling cavity
m mPR1 $recycR1 $recycT1 0 nPR0 nPR1
s sPRcav $Lrecyc nPR1 nPR2 
m mPR2 $recycR2 $recycT2 0 nPR2 nPR3
s sPRtoBS 0 nPR3 nBSi # null length

# central beam splitter
bs BS 0.5 0.5 0 0 nBSi nBSr nBSt nBSo
s sBStoYarm 0 nBSr nY0 # null length
s sBStoXarm 0 nBSt nX0 # null length
s sBStoSR 0 nBSo nSR0 # null length

# Y arm (perpendicular)
m mY1 $R1arm $T1arm 0 nY0 nY1
s sY $LY nY1 nY2
m mY2 $R2arm $T2arm 0 nY2 nY3

# X arm (parallel)
m mX1 $R1arm $T1arm 0 nX0 nX1
s sX $LX nX1 nX2
m mX2 $R2arm $T2arm 0 nX2 nX3

# signal recycling cavity
m mSR1 $recycR1 $recycT1 0 nSR0 nSR1
s sSRcav $Lrecyc nSR1 nSR2
m mSR2 $recycR2 $recycT2 0 nSR2 nSR3
s sSRtosqzBS 0 nSR3 nsqzBS1 # null length

# squeezer addition beam splitter
bs sqzBS 0.5 0.5 0 0 nsqzBS1 nsqzBSdump nsqzBSpd nNLEo
nle nle1 0.05 0 nNLEi nNLEo # r = 0.05, sqz angle = 0

# photodiode
pd darkport nsqzBSpd
"""
basekat.parse(basecode)

In [ ]:
pprint((basekat.components, basekat.detectors, basekat.commands))

In [ ]:
kat1 = deepcopy(basekat)
kat1code = """
fsig noise 1 # noise at 1Hz
xaxis noise f lin 1 10G 1000
#yaxis abs:deg
"""
kat1.parse(kat1code)
out1 = kat1.run()